# Explore confidence scores with Whisper

In [1]:
from utils import *
from confidence_utils import *

from datasets import load_dataset
from IPython.display import HTML as html_print

/Users/antonin/Documents/Projects/capstone/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Load data

In [2]:
fleurs_en = load_dataset("google/fleurs", "en_us")
fleurs_en = fleurs_en.remove_columns(['id', 'num_samples', 'path', 'gender', 'lang_id', 'language', 'lang_group_id'])
fleurs_en = fleurs_en['train'].select(range(10))

Found cached dataset fleurs (/Users/antonin/.cache/huggingface/datasets/google___fleurs/en_us/2.0.0/aabb39fb29739c495517ac904e2886819b6e344702f0a5b5283cb178b087c94a)
100%|██████████| 3/3 [00:00<00:00, 90.94it/s]


## 2. Load models

In [3]:
processor_en, model_en = load_whisper_with_confidence_scores('openai/whisper-base', 'English')

## 3. Run inference test

In [4]:
model_en.config.forced_decoder_ids = processor_en.get_decoder_prompt_ids(language = "en", task = "transcribe")
fleurs_en = fleurs_en.map(map_to_pred_and_confidence_scores, 
    fn_kwargs={"processor": processor_en, "model": model_en, "lang": "en"}, 
    batched=True, \
    remove_columns=['audio'], 
    batch_size = 1)
fleurs_en

Loading cached processed dataset at /Users/antonin/.cache/huggingface/datasets/google___fleurs/en_us/2.0.0/aabb39fb29739c495517ac904e2886819b6e344702f0a5b5283cb178b087c94a/cache-0ae6043dece4a134.arrow


Dataset({
    features: ['transcription', 'raw_transcription', 'string_pred', 'tokens_pred', 'probs_tokens_pred', 'ground_truth', 'wer'],
    num_rows: 10
})

## 4. Save processed data with confidence indicators

In [5]:
fleurs_en.save_to_disk(os.path.join(predictions_confidence_path, 'fleurs_en'))

## 5. Load processed data with confidence indicators

In [6]:
fleurs_en = load_from_disk(os.path.join(predictions_confidence_path, 'fleurs_en'))
fleurs_en

Dataset({
    features: ['transcription', 'raw_transcription', 'string_pred', 'tokens_pred', 'probs_tokens_pred', 'ground_truth', 'wer'],
    num_rows: 10
})

## 6. Display confidence colors

In [7]:
html_print(html_display_confidence(fleurs_en, range(3)))

## 7. Time to analyze

In [16]:
### TODO